### Create text8.txt

In [2]:
import numpy as np
import re
import gensim.downloader as api
from gensim.models import Word2Vec
dataset = api.load("text8")
model = Word2Vec(dataset)
model.save("data/text8-word2vec.bin")

path = dataset.fn
import os

# Abrir o arquivo "text8" para leitura de bytes
with open('./data/text8', 'rb') as file:
    content_bytes = file.read()

# Decodificar os bytes para uma string
content = content_bytes.decode('utf-8')

# Salvar o conteúdo em um novo arquivo de texto
with open('./data/text8_decoded.txt', 'w', encoding='utf-8') as file:
    file.write(content)



KeyboardInterrupt



In [3]:
with open('./data/text8_decoded.txt', 'r') as file:
    doc = file.read().lower()


In [4]:
corpus_list = re.split('\W+', doc)

In [5]:
len(corpus_list)

17005208

In [6]:
cutOffValue = 100
from collections import defaultdict
frequency = defaultdict(int)
for token in corpus_list:
    frequency[token]+= 1
processed_corpus_list = [token for token in corpus_list if frequency[token] >= cutOffValue]

In [7]:
len(processed_corpus_list)

15471435

In [8]:
allWords = np.array(list(frequency.keys()))
allCounts = np.array(list(frequency.values()))

In [9]:
vocab = allWords[allCounts >= cutOffValue]
wordCounts  = allCounts[allCounts >= cutOffValue]

In [10]:
from scipy.sparse import lil_matrix

In [11]:
def computeWordContextMatrix(corpus_list, vocab=None, window_size=2):
    if vocab is None:
        vocab = sorted(list(set(corpus_list)))
    numWords = len(vocab)  
    M = np.zeros((numWords, numWords))
#     M = lil_matrix((numWords, numWords))
    w_2_i_mapping = dict(zip(vocab, np.arange(numWords)))
    i_2_w_mapping = dict(zip(np.arange(numWords), vocab))
    doc = corpus_list
    curIdx = 0
    docLength = len(doc)
    while curIdx < docLength:
        left = max(curIdx - window_size, 0)
        right = min(curIdx + window_size + 1, docLength)
        wordsInContext = doc[left:curIdx] + doc[curIdx+1:right]
        currentWord = doc[curIdx]
        currentWordIdx = w_2_i_mapping[currentWord]
        for word in wordsInContext:
            contextWordIdx = w_2_i_mapping[word]
            M[currentWordIdx,contextWordIdx] += 1
        curIdx += 1
    return M, w_2_i_mapping, i_2_w_mapping

In [12]:
M, w_2_i_mapping, i_2_w_mapping = computeWordContextMatrix(processed_corpus_list, vocab)

In [83]:
M, w_2_i_mapping, i_2_w_mapping = computeWordContextMatrix(processed_corpus_list, vocab)

In [71]:
M.shape

(11815, 11815)

In [ ]:
len(vocab)

In [74]:
word = 'good'
print(w_2_i_mapping[word], i_2_w_mapping[190])

190 good


In [78]:
v = M[w_2_i_mapping[word]]
print(v)

[  0.   0. 214. ...   0.   0.   0.]


In [79]:
v.shape

(11815,)

In [13]:
def pmi(M, positive = True):
    col_totals = np.sum(M, axis=0)
    total = col_totals.sum()
    row_totals = np.sum(M, axis =1)
    expected = np.outer(row_totals, col_totals) / total
    M = M / expected
    with np.errstate(divide = 'ignore'):
        M = np.log(M)
    M[np.isinf(M)] = 0.0
    if positive:
        M[M<0] = 0.0
    return M

In [14]:
M = pmi(M)

In [86]:
M[w_2_i_mapping['good'], :]

matrix([[0.       , 0.       , 0.4605286, ..., 0.       , 0.       ,
         0.       ]])

### SVD2

In [1]:
from sklearn.decomposition import TruncatedSVD, PCA, IncrementalPCA

In [15]:
transformer = TruncatedSVD(n_components=100)

In [16]:
M_reduced = transformer.fit_transform(M)

In [18]:
M_reduced.shape


(11815, 100)